In [1]:
import os
import json
import os
import sys
import pandas as pd

notebook_dir = os.getcwd()
sys.path.append(os.path.abspath(os.path.join(notebook_dir, '..')))

from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from utils.s3 import get_s3_client, read_pdf_from_s3, write_dataframe_to_s3
from utils.azure_extract import extract_figures

In [3]:

url = 'https://sfopenaccessbucket.s3.us-east-1.amazonaws.com/uploads/Snowflake_Components_Installation_Guide.pdf'

pdf_data = read_pdf_from_s3(get_s3_client(), url)

In [4]:
parent_file = url.split('/uploads/')[1]

In [5]:
with open('data.json', 'r') as f:
    data = json.load(f)

In [6]:
s3_client = get_s3_client()

In [8]:
log=[]

In [9]:
for d in data['figures']:
    _p = d['id'].split('.')
    page_num, fig_num = int(_p[0]), int(_p[1])
    _l = extract_figures(s3_client=s3_client,
                         pdf_data=pdf_data,
                         parent_file=parent_file,
                         page_num=page_num,
                         id=fig_num,
                         polygon=d['boundingRegions'][0]['polygon']
                         ) 
    print(_l)
    log.append(f'Write (Image) : {_l}')   
    # print(d)

https://sfopenaccessbucket.s3.us-east-1.amazonaws.com/results/Snowflake_Components_Installation_Guide.pdf/1/images/1.jpeg
https://sfopenaccessbucket.s3.us-east-1.amazonaws.com/results/Snowflake_Components_Installation_Guide.pdf/1/images/2.jpeg
https://sfopenaccessbucket.s3.us-east-1.amazonaws.com/results/Snowflake_Components_Installation_Guide.pdf/2/images/1.jpeg
https://sfopenaccessbucket.s3.us-east-1.amazonaws.com/results/Snowflake_Components_Installation_Guide.pdf/3/images/1.jpeg
https://sfopenaccessbucket.s3.us-east-1.amazonaws.com/results/Snowflake_Components_Installation_Guide.pdf/4/images/1.jpeg
https://sfopenaccessbucket.s3.us-east-1.amazonaws.com/results/Snowflake_Components_Installation_Guide.pdf/4/images/2.jpeg


In [31]:
extract = lambda x: [i['content'] for i in x]
t_index=dict()
for table in data['tables']:
    # _p = d['id'].split('.')
    # page_num, fig_num = int(_p[0]), int(_p[1])
    # print(_l)
    page_num = table['cells'][0]['boundingRegions'][0]['pageNumber'] 
    if page_num not in t_index:
        t_index[page_num]=1
    _table=[]       
    for _ in range(0,len(table['cells']), table['columnCount']):
        _table.append(extract(table['cells'][_ : _+table['columnCount']]))
    df = pd.DataFrame(_table[1:], columns=_table[0])
    _l = write_dataframe_to_s3(s3_client=s3_client,
                            df=df,
                            parent_file=parent_file,
                            page_num=page_num,
                            id=t_index[page_num]
                            )
    t_index[page_num] = t_index[page_num]+1
    log.append(f'Write (Table) : {_l}')   


In [32]:
log

['Write (Image) : https://sfopenaccessbucket.s3.us-east-1.amazonaws.com/results/Snowflake_Components_Installation_Guide.pdf/1/images/1.jpeg',
 'Write (Image) : https://sfopenaccessbucket.s3.us-east-1.amazonaws.com/results/Snowflake_Components_Installation_Guide.pdf/1/images/2.jpeg',
 'Write (Image) : https://sfopenaccessbucket.s3.us-east-1.amazonaws.com/results/Snowflake_Components_Installation_Guide.pdf/2/images/1.jpeg',
 'Write (Image) : https://sfopenaccessbucket.s3.us-east-1.amazonaws.com/results/Snowflake_Components_Installation_Guide.pdf/3/images/1.jpeg',
 'Write (Image) : https://sfopenaccessbucket.s3.us-east-1.amazonaws.com/results/Snowflake_Components_Installation_Guide.pdf/4/images/1.jpeg',
 'Write (Image) : https://sfopenaccessbucket.s3.us-east-1.amazonaws.com/results/Snowflake_Components_Installation_Guide.pdf/4/images/2.jpeg',
 'Write (Table) : https://sfopenaccessbucket.s3.us-east-1.amazonaws.com/results/Snowflake_Components_Installation_Guide.pdf/2/tables/1.csv',
 'Write